# Ubuntu Setup Notes

I largely followed this [post](https://blog.slavv.com/the-1700-great-deep-learning-box-assembly-setup-and-benchmarks-148c5ebe6415) for the installation and software configuration instructions. 

A few additions are needed, perhaps specifically for the hardware I'm using.

## Ubuntu Package Installation

Refer to [here](https://askubuntu.com/questions/40779/how-do-i-install-a-deb-file-via-the-command-line) for installing packages. 

To install a `.deb` package:

    sudo dpkg -i DEB_PACKAGE

To resolve package dependency issues:

    sudo apt-get install -f
    
To remove a package:

    sudo dpkg -r PACKAGE_NAME

## Nvidia Driver / GeForce 1080 Ti Founders

System:

Intel Xeon E5-2630v4
Asus X99-E WS motherboard
2x Nvidia GeForce 1080 Ti

A few issues I encountered here:


### Driver Installation Issues, version 381.22

Had problem with this installation, potentially due to the fact that it was a reinstall. In a reinstall, some GL libs are **skipped** and not installed. Solution was to add manual switches to force them to be reinstalled. 

However, I still cannot get the `drm` tools to work. 

The parameters I used:

    sudo ./NVIDIA-Linux-x86_64-381.22.run -install-libglvnd -glvnd-glx-client -glvnd-egl-client -no-drm

Also, I needed to **upgrade to the latest distribution linux kernel for the drivers to work**. For this, I followed the instructions [here](https://askubuntu.com/questions/196768/how-to-install-updates-via-command-line) to update the packages for the distrubtions. The commends needed:

    sudo apt-get update        # Fetches the list of available updates
    sudo apt-get upgrade       # Strictly upgrades the current packages
    sudo apt-get dist-upgrade  # Installs updates (new ones)
    

### Disable Secure Boot

Some posts suggested disabling secure book in BIOS helped with the `drm` issue, but this did not work for me.



### PCIe Bus ERROR

This seems to be a problem for Asus, X99 chipset & PLX switches issue, according to this [post](https://forums.geforce.com/default/topic/957456/geforce-drivers/gtx-1080-throwing-bad-tlp-pcie-bus-errors/2/). 

I used one of the suggestions by adding this kernel parameters: `pcie_aspm=off`. This seems to have worked for me so far, no more error messages.

## Software Updates

### `xarray` for Anaconda

    conda install -c conda-forge xarray cartopy pynio

### Rethinking / R

Create `.Rprofile` file and write the line below for MRO.

    options(repos = c(CRAN = "https://cran.revolutionanalytics.com"))
    
`rstan` Installation [here](https://github.com/stan-dev/rstan/wiki/Installing-RStan-on-Mac-or-Linux)

    install.packages("rstan", repos = "https://cloud.r-project.org/", dependencies=TRUE)
    
Installing `Rethinking`:

    install.packages(c("coda","mvtnorm","devtools","loo"))
    library(devtools)
    devtools::install_github("rmcelreath/rethinking")